In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import nltk


In [ ]:
#pip install texthero

In [ ]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import random
import seaborn as sns
#import texthero as hero
#from texthero import preprocessing
import nltk
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from nltk.stem import WordNetLemmatizer

# BERT ON BALANCED DATASET

In [ ]:
df = pd.read_csv('../input/dataset/oversampled_preprocessed.csv')


In [ ]:
#len(df)

In [ ]:
df = df.sample(frac = 1)
df.head()

In [ ]:
seq_len =512
num_samples=len(df)

Xids=np.zeros((num_samples,seq_len))
Xmask=np.zeros((num_samples,seq_len))

Xids.shape

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

for i,sentence in enumerate(df['cleaned_description']):
    tokens=tokenizer.encode_plus(sentence,max_length=seq_len,truncation=True,
                                padding='max_length',add_special_tokens=True,
                                return_tensors='tf')
    Xids[i, :]=tokens['input_ids']
    Xmask[i, :]=tokens['attention_mask']
    

In [ ]:
Xids

In [ ]:
Xmask 

In [ ]:
df["classification_category"] = df["classification_category"].astype('category')
df['code'] = df.classification_category.cat.codes
df.head()

In [ ]:
import gc
arr = df['code'].values
arr
del df
gc.collect()


In [ ]:
labels = np.zeros((num_samples,arr.max()+1))
labels.shape

In [ ]:
labels[np.arange(num_samples),arr]=1

In [ ]:
labels

In [ ]:
import tensorflow as tf


In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((Xids,Xmask,labels))

dataset.take(1)

In [ ]:
def map_func(input_ids,masks,labels):
    return{'input_ids':input_ids,'attention_mask':masks},labels

In [ ]:
dataset=dataset.map(map_func)

In [ ]:
dataset.take(1)

In [ ]:
batch_size=8

dataset=dataset.shuffle(4000).batch(batch_size,drop_remainder=True)
dataset.take(1)


In [ ]:
split=0.8
size =int((num_samples / batch_size)*split)


In [ ]:
train_ds=dataset.take(size)
val_ds=dataset.skip(size)

del dataset

In [ ]:
from transformers import TFAutoModel

bert = TFAutoModel.from_pretrained('bert-base-uncased')

bert.summary()

In [ ]:
input_ids =tf.keras.layers.Input(shape=(seq_len,),name='input_ids',dtype='int32')
mask = tf.keras.layers.Input(shape=(seq_len,),name='attention_mask',dtype='int32')

embeddings=bert.bert(input_ids,attention_mask=mask)[1]

x=tf.keras.layers.Dense(1024,activation='relu')(embeddings)
y=tf.keras.layers.Dense(arr.max()+1,activation='softmax',name='outputs')(x)

In [ ]:
model=tf.keras.Model(inputs=[input_ids,mask],outputs=y)


model.summary()

In [ ]:
optimizer=tf.keras.optimizers.Adam(lr=1e-5,decay=1e-6)
loss= tf.keras.losses.CategoricalCrossentropy()
acc=tf.keras.metrics.CategoricalAccuracy('accuracy')

In [ ]:
model.compile(optimizer=optimizer,loss=loss,metrics=[acc])

In [ ]:
history=model.fit(train_ds,validation_data=val_ds,
                 epochs=3
                 )

# BERT ON UNBALANCED DATASET

In [ ]:
unbalanced_df = pd.read_csv('../input/dataset/unsampled_preprocessed.csv')

In [ ]:
unbalanced_df = unbalanced_df.sample(frac = 1)
unbalanced_df.head()

In [ ]:
seq_len =512
num_samples=len(df)

Xids=np.zeros((num_samples,seq_len))
Xmask=np.zeros((num_samples,seq_len))

Xids.shape

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

for i,sentence in enumerate(unbalanced_df['cleaned_description']):
    tokens=tokenizer.encode_plus(sentence,max_length=seq_len,truncation=True,
                                padding='max_length',add_special_tokens=True,
                                return_tensors='tf')
    Xids[i, :]=tokens['input_ids']
    Xmask[i, :]=tokens['attention_mask']
    

In [ ]:
Xids

In [ ]:
Xmask 

In [ ]:
unbalanced_df["classification_category"] = unbalanced_df["classification_category"].astype('category')
unbalanced_df['code'] = unbalanced_df.classification_category.cat.codes
unbalanced_df.head()

In [ ]:
import gc
arr = unbalanced_df['code'].values
arr
del unbalanced_df
gc.collect()

In [ ]:
labels = np.zeros((num_samples,arr.max()+1))
labels.shape

In [ ]:
labels[np.arange(num_samples),arr]=1

In [ ]:
import tensorflow as tf

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((Xids,Xmask,labels))

dataset.take(1)

In [ ]:
def map_func(input_ids,masks,labels):
    return{'input_ids':input_ids,'attention_mask':masks},labels

In [ ]:
dataset=dataset.map(map_func)

In [ ]:
dataset.take(1)

In [ ]:
batch_size=8

dataset=dataset.shuffle(4000).batch(batch_size,drop_remainder=True)
dataset.take(1)

In [ ]:
split=0.8
size =int((num_samples / batch_size)*split)

In [ ]:
train_ds=dataset.take(size)
val_ds=dataset.skip(size)

del dataset

In [ ]:
from transformers import TFAutoModel

bert = TFAutoModel.from_pretrained('bert-base-uncased')

bert.summary()

In [ ]:
input_ids =tf.keras.layers.Input(shape=(seq_len,),name='input_ids',dtype='int32')
mask = tf.keras.layers.Input(shape=(seq_len,),name='attention_mask',dtype='int32')

embeddings=bert.bert(input_ids,attention_mask=mask)[1]

x=tf.keras.layers.Dense(1024,activation='relu')(embeddings)
y=tf.keras.layers.Dense(arr.max()+1,activation='softmax',name='outputs')(x)

In [ ]:
model=tf.keras.Model(inputs=[input_ids,mask],outputs=y)


model.summary()

In [ ]:
optimizer=tf.keras.optimizers.Adam(lr=1e-5,decay=1e-6)
loss= tf.keras.losses.CategoricalCrossentropy()
acc=tf.keras.metrics.CategoricalAccuracy('accuracy')

In [ ]:
model.compile(optimizer=optimizer,loss=loss,metrics=[acc])

In [ ]:
history=model.fit(train_ds,validation_data=val_ds,
                 epochs=3
                 )